In [31]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

In [4]:
data_storage_directory = r"C:\temp"  # Note: this path must exist on your local drive
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

ophys_session_table.csv: 100%|█████████████████████████████████████████████████████| 227k/227k [00:00<00:00, 1.04MMB/s]
behavior_session_table.csv: 100%|████████████████████████████████████████████████| 1.21M/1.21M [00:00<00:00, 8.18MMB/s]
ophys_experiment_table.csv: 100%|██████████████████████████████████████████████████| 610k/610k [00:00<00:00, 4.24MMB/s]
ophys_cells_table.csv: 100%|█████████████████████████████████████████████████████| 4.29M/4.29M [00:00<00:00, 20.1MMB/s]


In [5]:
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()

In [6]:
session_sst = session_table[['Sst' in session_table.loc[i].driver_line[0] for i in session_table.index]]
session_slc = session_table[['Slc' in session_table.loc[i].driver_line[0] for i in session_table.index]]
session_vip = session_table[['Vip' in session_table.loc[i].driver_line[0] for i in session_table.index]]

print('SST', len(session_sst))
print('Slc', len(session_slc))
print('Vip', len(session_vip))

SST 158
Slc 361
Vip 186


In [7]:
sst_sessions = np.unique(np.array(session_sst.index))
session_sst.head()

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,behavior_session_id,ophys_experiment_id,ophys_container_id,project_code,date_of_acquisition,session_type
ophys_session_id,,,,,,,,,,,,,,,,,,,
951410079,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0.0,65.0,0.0,951520319,"[951980471, 951980473, 951980475, 951980479, 9...","[1018028360, 1018028345, 1018028357, 101802834...",VisualBehaviorMultiscope,2019-09-20 09:45:29.897856,OPHYS_1_images_A
952430817,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,209.0,Sst-IRES-Cre,GCaMP6f,2.0,0.0,66.0,1.0,952554548,"[953659743, 953659745, 953659749, 953659752]","[1018028357, 1018028345, 1018028339, 1018028354]",VisualBehaviorMultiscope,2019-09-23 08:13:07.627573,OPHYS_2_images_A_passive
954954402,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,210.0,Sst-IRES-Cre,GCaMP6f,3.0,0.0,67.0,2.0,953982960,"[958527464, 958527471, 958527474, 958527479, 9...","[1018028360, 1018028342, 1018028357, 101802835...",VisualBehaviorMultiscope,2019-09-24 16:00:00.000000,OPHYS_3_images_A
955775716,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,212.0,Sst-IRES-Cre,GCaMP6f,3.0,1.0,68.0,3.0,956010809,"[956941841, 956941844, 956941846]","[1018028342, 1018028339, 1018028345]",VisualBehaviorMultiscope,2019-09-26 08:44:37.199778,OPHYS_3_images_A
957020350,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,213.0,Sst-IRES-Cre,GCaMP6f,4.0,0.0,0.0,4.0,957032492,"[957759562, 957759564, 957759566, 957759570, 9...","[1018028354, 1018028345, 1018028360, 101802834...",VisualBehaviorMultiscope,2019-09-27 08:28:05.415005,OPHYS_4_images_B


# Load all data for one type of neuron, save each session

In [26]:
savingLoc = r'W:\People\Yannan\allenDataset\SST'

In [28]:

for ophys_session_id in [sst_sessions[0]]:

    neural_data = []
    neural_data_cleaned = pd.DataFrame({})
    
    ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
    for ophys_experiment_id in ophys_experiment_ids:
        this_experiment = cache.get_behavior_ophys_experiment(ophys_experiment_id)

        this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)

        # add some columns with metadata for the experiment
        metadata_keys = [
            'ophys_experiment_id',
            'ophys_session_id',
            'targeted_structure',
            'imaging_depth',
            'equipment_name',
            'cre_line',
            'mouse_id',
            'sex',
        ]
        for metadata_key in metadata_keys:
            this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

#         # append the data for this experiment to a list
#         neural_data.append(this_experiment_neural_data)

#     # concatate the list of dataframes into a single dataframe
#     neural_data = pd.concat(neural_data)
    
    
        all_cell_specimen_id = np.unique(this_experiment_neural_data.cell_specimen_id)

        
        for cell_specimen_id in all_cell_specimen_id:
            cell_data = this_experiment_neural_data.loc[this_experiment_neural_data.cell_specimen_id == cell_specimen_id]
            neural_data_cleaned_single = pd.concat((pd.DataFrame({
              'timestamps': [cell_data.timestamps],
              'dff': [cell_data.dff],
              'events': [cell_data.events],
              'filtered_events': [cell_data.filtered_events],
            }), pd.DataFrame(cell_data.iloc[0,4:]).transpose()), axis = 1)
            neural_data_cleaned = pd.concat([neural_data_cleaned, neural_data_cleaned_single], axis = 0)

    neural_data_cleaned.to_pickle(savingLoc + r'\{}.pkl'.format(ophys_session_id))

In [29]:
help(mindscope_utilities.event_triggered_response)

Help on function event_triggered_response in module mindscope_utilities.general_utilities:

event_triggered_response(data, t, y, event_times, t_start=None, t_end=None, t_before=None, t_after=None, output_sampling_rate=None, include_endpoint=True, output_format='tidy', interpolate=True)
    Slices a timeseries relative to a given set of event times
    to build an event-triggered response.
    
    For example, If we have data such as a measurement of neural activity
    over time and specific events in time that we want to align
    the neural activity to, this function will extract segments of the neural
    timeseries in a specified time window around each event.
    
    The times of the events need not align with the measured
    times of the neural data.
    Relative times will be calculated by linear interpolation.
    
    Parameters:
    -----------
    data: Pandas.DataFrame
        Input dataframe in tidy format
        Each row should be one observation
        Must contains

In [32]:
ophys_session_id

1110797833

In [39]:
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
experiments = cache.get_behavior_ophys_experiment(ophys_experiment_ids[0])

In [40]:
experiments = {}
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
for ophys_experiment_id in ophys_experiment_ids:
    experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)

In [42]:
stimulus_table = experiments[ophys_experiment_ids[0]].stimulus_presentations.drop(columns = ['image_set'])  # dropping the 'image_set' column to avoid confusion. Image_set column contains a unique string for set of images presented in a session.
stimulus_table.head(10)

,start_time,stop_time,duration,image_name,image_index,is_change,omitted,start_frame,end_frame
stimulus_presentations_id,,,,,,,,,
0,309.68939,309.93960,0.25021,im104_r,0,False,False,17986,18001.0
1,310.44002,310.69022,0.25020,im104_r,0,False,False,18031,18046.0
2,311.19063,311.44084,0.25021,im104_r,0,False,False,18076,18091.0
3,311.94125,312.19145,0.25020,im104_r,0,False,False,18121,18136.0
4,312.69186,312.94206,0.25020,im104_r,0,False,False,18166,18181.0
5,313.44246,313.69272,0.25026,im104_r,0,False,False,18211,18226.0
6,314.19311,314.44329,0.25018,im104_r,0,False,False,18256,18271.0
7,314.94371,315.19387,0.25016,im104_r,0,False,False,18301,18316.0
8,315.69427,315.94447,0.25020,im104_r,0,False,False,18346,18361.0


In [43]:

ophys_session_id = sst_sessions[0]

neural_data = []
neural_data_cleaned = pd.DataFrame({})

ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']

ophys_experiment_id = ophys_experiment_ids[0]

this_experiment = cache.get_behavior_ophys_experiment(ophys_experiment_id)

this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)

# add some columns with metadata for the experiment
metadata_keys = [
    'ophys_experiment_id',
    'ophys_session_id',
    'targeted_structure',
    'imaging_depth',
    'equipment_name',
    'cre_line',
    'mouse_id',
    'sex',
]
for metadata_key in metadata_keys:
    this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

stimulus_table = this_experiment.stimulus_presentations.drop(columns = ['image_set'])  # dropping the 'image_set' column to avoid confusion. Image_set column contains a unique string for set of images presented in a session.

#         # append the data for this experiment to a list
#         neural_data.append(this_experiment_neural_data)

#     # concatate the list of dataframes into a single dataframe
#     neural_data = pd.concat(neural_data)


# all_cell_specimen_id = np.unique(this_experiment_neural_data.cell_specimen_id)


# for cell_specimen_id in all_cell_specimen_id:
#     cell_data = this_experiment_neural_data.loc[this_experiment_neural_data.cell_specimen_id == cell_specimen_id]
#     neural_data_cleaned_single = pd.concat((pd.DataFrame({
#       'timestamps': [cell_data.timestamps],
#       'dff': [cell_data.dff],
#       'events': [cell_data.events],
#       'filtered_events': [cell_data.filtered_events],
#     }), pd.DataFrame(cell_data.iloc[0,4:]).transpose()), axis = 1)
#     neural_data_cleaned = pd.concat([neural_data_cleaned, neural_data_cleaned_single], axis = 0)

# neural_data_cleaned.to_pickle(savingLoc + r'\{}.pkl'.format(ophys_session_id))

In [44]:
stimulus_table

,start_time,stop_time,duration,image_name,image_index,is_change,omitted,start_frame,end_frame
stimulus_presentations_id,,,,,,,,,
0,308.92068,309.17088,0.25020,im065,0,False,False,17986,18001.0
1,309.67130,309.92148,0.25018,im065,0,False,False,18031,18046.0
2,310.42190,310.67211,0.25021,im065,0,False,False,18076,18091.0
3,311.15584,311.40584,0.25000,omitted,8,False,True,18120,NaN
4,311.92309,312.17329,0.25020,im065,0,False,False,18166,18181.0
...,...,...,...,...,...,...,...,...,...
4805,3916.29897,3916.54897,0.25000,omitted,8,False,True,234252,NaN
4806,3917.06623,3917.31644,0.25021,im077,1,False,False,234298,234313.0
4807,3917.81683,3918.06704,0.25021,im077,1,False,False,234343,234358.0


In [45]:
[1,1,0] and [0,1,0]

[0, 1, 0]